In [ ]:
# Transformers installation
!pip install "ray[tune]" transformers datasets scipy sklearn torch
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 54.5 MB 297 kB/s 
     |████████████████████████████████| 4.7 MB 58.6 MB/s 
     |████████████████████████████████| 365 kB 73.7 MB/s 
     |████████████████████████████████| 596 kB 79.2 MB/s 
     |████████████████████████████████| 6.6 MB 74.3 MB/s 
     |████████████████████████████████| 101 kB 10.4 MB/s 
     |████████████████████████████████| 115 kB 70.5 MB/s 
     |████████████████████████████████| 212 kB 64.0 MB/s 
     |████████████████████████████████| 141 kB 75.6 MB/s 
     |████████████████████████████████| 127 kB 71.6 MB/s 
     |████████████████████████████████| 8.8 MB 50.1 MB/s 
     |████████████████████████████████| 4.1 MB 62.2 MB/s 
     |████████████████████████████████| 125 kB 95.7 MB/s 
     |████████████████████████████████| 466 kB 78.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Unins

In [ ]:
!pip install wandb -qq

In [ ]:
!wandb login

wandb: Currently logged in as: bobliu. Use `wandb login --relogin` to force relogin


In [ ]:
from datasets import load_dataset, load_metric
from transformers import (AutoModelForSequenceClassification, AutoTokenizer,
                          Trainer, TrainingArguments)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

def compute_metrics_old(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=-1)
    return metric.compute(predictions=predictions, references=labels)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

def model_init():
    return AutoModelForSequenceClassification.from_pretrained("GroNLP/hateBERT", num_labels=5)

dataset = load_dataset("classla/FRENK-hate-en")
dataset.remove_columns(['target', 'topic'])
tokenizer = AutoTokenizer.from_pretrained("GroNLP/hateBERT")
tokenized_datasets = dataset.map(tokenize_function, batched=True)

small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(100))

training_args = TrainingArguments(output_dir="test_trainer", 
                                  evaluation_strategy="epoch", 
                                  save_strategy='epoch',
                                  load_best_model_at_end=True, 
                                  per_device_train_batch_size=8, 
                                  report_to="wandb")

trainer = Trainer(
    model_init=model_init,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics
)

# Default objective is the sum of all metrics, so we have to maximize it.
trainer.hyperparameter_search(
    direction="maximize", 
    backend="ray", 
    n_trials=10 
)

trainer.train()


  0%|          | 0/3 [00:00<?, ?it/s]

loading configuration file https://huggingface.co/GroNLP/hateBERT/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/beaf3bf77311e5d01ddc7e50c577f9c2a26c55a8ec8ea5f8c81f755acdeebc10.53b298b0646108c5ea759a24a4d397baeea68c6d2844015cb0a6f6d4eda0cb67
Model config BertConfig {
  "_name_or_path": "GroNLP/hateBERT",
  "_num_labels": 2,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/GroNLP/hateBERT/resolve/

== Status ==
Current time: 2022-08-11 15:28:32 (running for 00:00:00.24)
Memory usage on this node: 2.2/12.7 GiB
Using FIFO scheduling algorithm.
Resources requested: 1.0/2 CPUs, 0/0 GPUs, 0.0/7.34 GiB heap, 0.0/3.67 GiB objects
Result logdir: /root/ray_results/_objective_2022-08-11_15-28-31
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+------------------------+----------+----------------+-----------------+--------------------+-------------------------------+----------+
| Trial name             | status   | loc            |   learning_rate |   num_train_epochs |   per_device_train_batch_size |     seed |
|------------------------+----------+----------------+-----------------+--------------------+-------------------------------+----------|
| _objective_41c77_00000 | RUNNING  | 172.28.0.2:540 |     5.61152e-06 |                  5 |                            64 |  8.15396 |
| _objective_41c77_00001 | PENDING  |                |     1.56207e-05 |                  2 |                   

(_objective pid=540) Some weights of the model checkpoint at GroNLP/hateBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias']
(_objective pid=540) - This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(_objective pid=540) - This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(_objective pid=540) Some weights of BertForSequenceClassifica

== Status ==
Current time: 2022-08-11 15:28:37 (running for 00:00:05.82)
Memory usage on this node: 2.5/12.7 GiB
Using FIFO scheduling algorithm.
Resources requested: 2.0/2 CPUs, 0/0 GPUs, 0.0/7.34 GiB heap, 0.0/3.67 GiB objects
Result logdir: /root/ray_results/_objective_2022-08-11_15-28-31
Number of trials: 10/10 (8 PENDING, 2 RUNNING)
+------------------------+----------+----------------+-----------------+--------------------+-------------------------------+----------+
| Trial name             | status   | loc            |   learning_rate |   num_train_epochs |   per_device_train_batch_size |     seed |
|------------------------+----------+----------------+-----------------+--------------------+-------------------------------+----------|
| _objective_41c77_00000 | RUNNING  | 172.28.0.2:540 |     5.61152e-06 |                  5 |                            64 |  8.15396 |
| _objective_41c77_00001 | RUNNING  | 172.28.0.2:576 |     1.56207e-05 |                  2 |                   

...

== Status ==
Current time: 2022-08-11 15:30:05 (running for 00:01:33.54)
Memory usage on this node: 4.5/12.7 GiB
Using FIFO scheduling algorithm.
Resources requested: 2.0/2 CPUs, 0/0 GPUs, 0.0/7.34 GiB heap, 0.0/3.67 GiB objects
Result logdir: /root/ray_results/_objective_2022-08-11_15-28-31
Number of trials: 10/10 (2 ERROR, 6 PENDING, 2 RUNNING)
+------------------------+----------+----------------+-----------------+--------------------+-------------------------------+----------+
| Trial name             | status   | loc            |   learning_rate |   num_train_epochs |   per_device_train_batch_size |     seed |
|------------------------+----------+----------------+-----------------+--------------------+-------------------------------+----------|
| _objective_41c77_00002 | RUNNING  | 172.28.0.2:770 |     8.28892e-06 |                  5 |                            16 | 24.4435  |
| _objective_41c77_00003 | RUNNING  | 172.28.0.2:811 |     1.09943e-06 |                  2 |          

2022-08-11 15:30:48,226	WARNING worker.py:1404 -- A worker died or was killed while executing a task by an unexpected system error. To troubleshoot the problem, check the logs for the dead worker. RayTask ID: ffffffffffffffffe8b78a7037fc589bd5f1273001000000 Worker ID: fee0660dd9bf062cada4f973d3befd2dd1a4c9f84293431d31b33797 Node ID: 4ee6c9df3f49d14f4671cb8c89e290bbcb3bd036a33bbfb7603faf27 Worker IP address: 172.28.0.2 Worker port: 42443 Worker PID: 770
2022-08-11 15:30:48,241	ERROR trial_runner.py:886 -- Trial _objective_41c77_00002: Error processing event.
NoneType: None


Result for _objective_41c77_00002:
  date: 2022-08-11_15-29-50
  experiment_id: 8be43fc6bcad4b7cb9e95f2e1bb99387
  hostname: f5c59a6c083e
  node_ip: 172.28.0.2
  pid: 770
  timestamp: 1660231790
  trial_id: 41c77_00002
  
== Status ==
Current time: 2022-08-11 15:30:51 (running for 00:02:18.94)
Memory usage on this node: 7.4/12.7 GiB
Using FIFO scheduling algorithm.
Resources requested: 2.0/2 CPUs, 0/0 GPUs, 0.0/7.34 GiB heap, 0.0/3.67 GiB objects
Result logdir: /root/ray_results/_objective_2022-08-11_15-28-31
Number of trials: 10/10 (3 ERROR, 5 PENDING, 2 RUNNING)
+------------------------+----------+----------------+-----------------+--------------------+-------------------------------+----------+
| Trial name             | status   | loc            |   learning_rate |   num_train_epochs |   per_device_train_batch_size |     seed |
|------------------------+----------+----------------+-----------------+--------------------+-------------------------------+----------|
| _objective_41c77

(_objective pid=976) Some weights of the model checkpoint at GroNLP/hateBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
(_objective pid=976) - This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(_objective pid=976) - This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(_objective pid=976) Some weights of BertForSequenceClassifica

== Status ==
Current time: 2022-08-11 15:31:13 (running for 00:02:41.60)
Memory usage on this node: 9.2/12.7 GiB
Using FIFO scheduling algorithm.
Resources requested: 2.0/2 CPUs, 0/0 GPUs, 0.0/7.34 GiB heap, 0.0/3.67 GiB objects
Result logdir: /root/ray_results/_objective_2022-08-11_15-28-31
Number of trials: 10/10 (3 ERROR, 5 PENDING, 2 RUNNING)
+------------------------+----------+----------------+-----------------+--------------------+-------------------------------+----------+
| Trial name             | status   | loc            |   learning_rate |   num_train_epochs |   per_device_train_batch_size |     seed |
|------------------------+----------+----------------+-----------------+--------------------+-------------------------------+----------|
| _objective_41c77_00003 | RUNNING  | 172.28.0.2:811 |     1.09943e-06 |                  2 |                             8 | 29.158   |
| _objective_41c77_00004 | RUNNING  | 172.28.0.2:976 |     2.3102e-06  |                  5 |          